# IBMi Jupyter Examples

Some examples of calling IBMi in Jupyter Notebooks

In [55]:
# Run this to initialize IBMi notebook
%run ../IBMi.ipynb

Enter host: DEV400
Enter user: OTTEB
Enter password: ········
Connecting to DEV400 as OTTEB ...
Successfully connected!


## List QSYS Tables

In [56]:
%%ibmi

select TABLE_SCHEMA || '/' || TABLE_NAME as TABLE_NAME, TABLE_TEXT
from QSYS2.SYSTABLES
where TABLE_SCHEMA='QSYS'
limit 3;

Executing statement 1 of 1 ...


,TABLE_NAME,TABLE_TEXT
0,QSYS/FMWINFO,Output file for DSPFMWSTS
1,QSYS/QAANZLOG,Output file for PRTERRLOG *ANZLOG
2,QSYS/QABNDBND,System supplied outfile for DSPBNDDIR command.


## List Members in a Source Physical File

In [57]:
%%ibmi

select TABLE_SCHEMA, TABLE_NAME, TABLE_PARTITION, SOURCE_TYPE
from QSYS2.SYSPARTITIONSTAT
where TABLE_SCHEMA='BOLIB'
  and TABLE_NAME='QRPGLESRC'
order by TABLE_PARTITION
limit 10;

Executing statement 1 of 1 ...


,TABLE_SCHEMA,TABLE_NAME,TABLE_PARTITION,SOURCE_TYPE
0,BOLIB,QRPGLESRC,BTCHASH,RPGLE
1,BOLIB,QRPGLESRC,CLR019B,SQLRPGLE
2,BOLIB,QRPGLESRC,EXCHAIN,RPGLE
3,BOLIB,QRPGLESRC,FIXEDR2,RPGLE
4,BOLIB,QRPGLESRC,FIXEDTEST,RPGLE
5,BOLIB,QRPGLESRC,FIZZBUZZ,RPGLE
6,BOLIB,QRPGLESRC,HELLOQSH,RPGLE
7,BOLIB,QRPGLESRC,HXCVT,RPGLE
8,BOLIB,QRPGLESRC,OV10MILL01,SQLRPGLE
9,BOLIB,QRPGLESRC,OV10MILL2,RPGLE


## View an RPGLE Source Member

In [58]:
%%ibmi 
create or replace alias QTEMP.TMPSRC1 for DEV400.BOLIB.QRPGLESRC(FIZZBUZZ);

select SRCDAT, SRCDTA
from QTEMP.TMPSRC1;

drop alias QTEMP.TMPSRC1;

Executing statement 1 of 3 ...
Statement executed successfully
Executing statement 2 of 3 ...


,SRCDAT,SRCDTA
0,190402.0,/free
1,190402.0,// The classic fizzbuzz program in RPGLE Free
2,190402.0,dcl-s num int(10);
3,190402.0,
4,190402.0,for num = 1 to 100;
5,190403.0,if %REM(num:3) = 0 and %REM(num:5) = 0;
6,190402.0,dsply ('num - ' + %char(num) + ' FIZZBUZZ');
7,190402.0,elseif %rem(num:3) = 0;
8,190402.0,dsply ('num - ' + %char(num) + ' FIZZ');
9,190402.0,elseif %rem(num:5) = 0;


Executing statement 3 of 3 ...
Statement executed successfully


## Calling an API with DB2

In [11]:
%%ibmi

select 
    id, 
    username, 
    url, 
    (minutes / 60) as hours_watched
from json_table(
    Systools.HttpPostClob(
        'https://graphql.anilist.co',
        cast('<httpHeader><header name="Content-Type" value="application/json"/></httpHeader>' as clob),
        cast('{"query": "{User(search:\"barrettotte\"){id name siteUrl stats{watchedTime}}}"}' as clob)
    ),
    '$.data.User'
    columns(
        id       char(10) path '$.id',
        username char(20) path '$.name',
        url      char(32) path '$.siteUrl',
        minutes  char(10) path '$.stats.watchedTime'
    )
);

Executing...


,ID,USERNAME,URL,HOURS_WATCHED
0,247578,barrettotte,https://anilist.co/user/247578,1639
